In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()


from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Reshape
from keras.layers import Dropout
import datetime

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Forecast_Dataset.xlsx to Forecast_Dataset.xlsx


In [3]:
n_lag_predizione = 14

n_input = 90 

In [4]:
df = pd.read_excel('Forecast_Dataset.xlsx')
df.fillna(method="ffill",inplace=True)
df.set_index(["DATA","Pranzo/Cena"],inplace=True)
df

A   B   C        D          E      F   G    H   I  \
DATA       Pranzo/Cena                                                        
2021-06-01 Cena         175  20  57   995.00   5979.000  16518  16  139  67   
           Pranzo       144  11  78   646.50   4213.360   8869  11  132  31   
2021-06-02 Cena         212  34  76  1096.00   7010.500  19643  25  141  94   
           Pranzo       184  11  63  1012.50   6362.476  12365  33  177  47   
2021-06-03 Cena          87  15  43   481.94   3419.677  14788  21  102  46   
...                     ...  ..  ..      ...        ...    ...  ..  ...  ..   
2022-03-20 Pranzo       320   9  76  1672.20  10077.424  18799  36  245  69   
2022-03-21 Cena         115   9  36   658.00   4131.660  11357  12   93  54   
           Pranzo       135  13  71   664.60   4027.872  11399   9  189  31   
2022-03-22 Cena         122  22  66   677.34   4656.388  16443  12  110  75   
           Pranzo       128  14  75   649.00   4196.380   8827   9  110  36   

                          J  ...   O  P    Q   R    S   T   U   V   W   X  
DATA       Pranzo/Cena       ...                                           
2021-06-01 Cena          75  ...   4  0  152  13   25  28  46  13  21  52  
           Pranzo       132  ...   7  4   58  37   26   7   8  11  21  52  
2021-06-02 Cena          56  ...  11  0  188  22   37  19  65  33  21  50  
           Pranzo        99  ...   9  0  123  45   45  19  25  25  21  50  
2021-06-03 Cena         102  ...  23  0   70  26   26  38   6  26  23  52  
...                     ...  ...  .. ..  ...  ..  ...  ..  ..  ..  ..  ..  
2022-03-20 Pranzo       160  ...   0  0  102  91  116  48  43  38   9  44  
2022-03-21 Cena          44  ...  10  0   99   5   15  11  31   7   9  43  
           Pranzo       154  ...   6  1   95  35   31  24  14  14   9  43  
2022-03-22 Cena          81  ...  23  0  120  12   25  16  27  24   8  47  
           Pranzo       120  ...  10  4   73  26   24  11  16  20   8  47  

[588 rows x 24 columns]

In [5]:
df = df.iloc[408:]
#df

In [6]:
df_train = df[:][:-n_lag_predizione]
df_test = df[:][-n_lag_predizione:]

In [ ]:
#df_train

---

In [7]:
n_features = len(df.columns)
train = df_train.to_numpy()
train = train.reshape(-1, n_features)

In [8]:
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)

In [9]:
scaled_train.shape

(166, 24)

In [10]:
# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            save_best_only=True,monitor='acc',mode='max',) # save only the best model to file

In [11]:
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=128)

In [12]:
models = []
for i in range(0,10):
  loss_func = "mse"
  model = Sequential(name="DNN_"+str(i))
  model.add(tf.keras.layers.Flatten())
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(rate=0.3))
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(rate=0.3))
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(rate=0.3))
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(rate=0.3))
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(rate=0.3))
  model.add(Dense(n_features))
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0015), loss=loss_func,metrics=['acc'])
  models.append(model)

In [13]:
models_df_pred = []
for m in models:
  m.fit(generator, epochs=1000, callbacks=[create_model_checkpoint(model_name=model.name)])
  m = tf.keras.models.load_model("model_experiments/"+model.name)
  test_predictions = []

  first_eval_batch = scaled_train[-n_input:]
  current_batch = first_eval_batch.reshape((1, n_input, n_features))
  for i in range(n_lag_predizione):
      current_pred = m.predict(current_batch)[0]
      test_predictions.append(current_pred) 
      current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
  true_predictions = scaler.inverse_transform(test_predictions)
  df_prediction = pd.DataFrame(data=true_predictions, index = df_test.index, columns=df_test.columns)
  df_prediction = df_prediction.apply(lambda x: round(x))
  models_df_pred.append(df_prediction)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 41ms/step - loss: 0.0042 - acc: 0.8026
Epoch 522/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.0041 - acc: 0.7632
Epoch 523/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.0050 - acc: 0.8158
Epoch 524/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.0039 - acc: 0.7500
Epoch 525/1000
1/1 [==============================] - 0s 46ms/step - loss: 0.0041 - acc: 0.8421
Epoch 526/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.0049 - acc: 0.8289
Epoch 527/1000
1/1 [==============================] - 0s 46ms/step - loss: 0.0061 - acc: 0.8026
Epoch 528/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.0047 - acc: 0.7632
Epoch 529/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.0042 - acc: 0.8026
Epoch 530/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.0059 - acc: 0.8421
Epoc

In [14]:
df_true = df[:][-n_lag_predizione:]

In [15]:
df_prediction_avg = models_df_pred[0]
for i in range(1,len(models_df_pred)):
  df_prediction_avg = df_prediction_avg + models_df_pred[i]

df_prediction_avg = df_prediction_avg/10

In [16]:
df_prediction_std = (models_df_pred[0]- df_prediction_avg)**2
for i in range(1,len(models_df_pred)):
  df_prediction_std = df_prediction_std + (models_df_pred[i] - df_prediction_avg)**2

df_prediction_std = (df_prediction_std/10)**(1/2)

In [17]:
interval = 1.96 * df_prediction_std

In [18]:
upper = df_prediction_avg + interval
lower = df_prediction_avg - interval

In [19]:
len(models_df_pred)

10

In [39]:
for colonna in df_prediction.columns:
    title = "Forecast : "+colonna
    fig = go.Figure()
    i = 1
    for df_model in models_df_pred:
      fig.add_trace(go.Scatter(x=[df_model.reset_index()["DATA"],df_model.reset_index()["Pranzo/Cena"]], y=df_model[colonna],
                          mode='markers',
                          line=dict(color='grey',width=2),
                          name='Forecast model '+str(i),
                          #text = df_prediction[colonna].apply(str)
                          ))
      i = i + 1
    #TRUE
    fig.add_trace(go.Scatter(x=[df_true.reset_index()["DATA"],df_true.reset_index()["Pranzo/Cena"]], y=df_true[colonna],
                        mode='lines+markers',
                        name='True',
                        line=dict(color='blue', width=2),
                        text = df_true[colonna].apply(str)))
    #AVG 
    fig.add_trace(go.Scatter(x=[df_prediction_avg.reset_index()["DATA"],df_prediction_avg.reset_index()["Pranzo/Cena"]], y=df_prediction_avg[colonna],
                        mode='lines+markers',
                        name='AVG forecast',
                        line=dict(color='green', width=4),
                        #fill='tonexty',
                        text = df_prediction_avg[colonna].apply(str)))
    #upper 
    fig.add_trace(go.Scatter(x=[upper.reset_index()["DATA"],upper.reset_index()["Pranzo/Cena"]], y=upper[colonna],
                        mode='lines+markers',
                        name='upper bound',
                        line=dict(color='green', width=1),
                        #fill='tonexty',
                        text = upper[colonna].apply(str)))
    #lower
    fig.add_trace(go.Scatter(x=[lower.reset_index()["DATA"],lower.reset_index()["Pranzo/Cena"]], y=lower[colonna],
                        mode='lines+markers',
                        name='lower bound',
                        line=dict(color='green', width=1),
                        fill='tonexty',
                        text = lower[colonna].apply(str)))
    
    fig.update_layout(title=title,
                       xaxis_title='Data',
                       yaxis_title='Valore')
    fig.update_traces(textposition="top center")

    fig.write_html(title+".html")
    fig.show()

In [22]:
df_in_bounds = (df_true < upper) & (df_true > lower)
df_in_bounds = df_in_bounds[df_in_bounds.columns[:-1]]
df_in_bounds

A      B      C      D      E      F      G  \
DATA       Pranzo/Cena                                                    
2022-03-16 Cena         False  False  False  False  False  False  False   
           Pranzo       False  False  False   True   True   True  False   
2022-03-17 Cena          True  False   True  False  False  False  False   
           Pranzo       False  False  False  False  False  False  False   
2022-03-18 Cena         False  False  False  False  False  False  False   
           Pranzo        True   True  False  False  False  False  False   
2022-03-19 Cena         False   True  False  False  False   True  False   
           Pranzo       False   True  False  False   True  False  False   
2022-03-20 Cena         False  False  False  False  False  False   True   
           Pranzo       False   True  False  False  False   True   True   
2022-03-21 Cena          True  False   True  False  False   True   True   
           Pranzo       False   True  False  False   True  False   True   
2022-03-22 Cena         False  False   True  False  False   True   True   
           Pranzo        True   True   True  False  False  False   True   

                            H      I      J  ...      N      O      P      Q  \
DATA       Pranzo/Cena                       ...                               
2022-03-16 Cena         False  False  False  ...  False  False   True  False   
           Pranzo       False  False  False  ...  False  False   True   True   
2022-03-17 Cena         False   True  False  ...   True  False  False  False   
           Pranzo       False  False  False  ...  False  False  False  False   
2022-03-18 Cena         False  False  False  ...  False  False  False  False   
           Pranzo       False  False  False  ...  False   True  False  False   
2022-03-19 Cena          True  False  False  ...  False   True   True   True   
           Pranzo       False  False   True  ...   True   True  False  False   
2022-03-20 Cena         False  False  False  ...  False   True  False  False   
           Pranzo       False   True  False  ...   True   True  False   True   
2022-03-21 Cena          True  False   True  ...  False  False  False  False   
           Pranzo       False   True  False  ...   True  False  False  False   
2022-03-22 Cena         False   True   True  ...  False  False  False  False   
           Pranzo       False   True  False  ...   True   True   True  False   

                            R      S      T      U      V      W  
DATA       Pranzo/Cena                                            
2022-03-16 Cena         False  False  False   True  False  False  
           Pranzo        True  False  False   True  False  False  
2022-03-17 Cena         False   True  False   True  False  False  
           Pranzo       False   True  False  False   True  False  
2022-03-18 Cena          True  False  False  False  False  False  
           Pranzo       False   True   True   True  False  False  
2022-03-19 Cena         False  False  False  False   True  False  
           Pranzo        True  False  False  False  False  False  
2022-03-20 Cena         False   True  False  False   True  False  
           Pranzo       False  False   True   True   True  False  
2022-03-21 Cena         False   True  False  False  False  False  
           Pranzo        True  False  False   True   True  False  
2022-03-22 Cena          True   True  False  False   True   True  
           Pranzo        True   True   True  False  False   True  

[14 rows x 23 columns]

In [23]:
px.imshow(df_in_bounds, aspect="auto", text_auto=True)

In [24]:
df_in_bounds.sum()

A    4
B    6
C    4
D    1
E    3
F    5
G    6
H    2
I    5
J    3
K    2
L    4
M    5
N    5
O    6
P    4
Q    3
R    6
S    7
T    3
U    6
V    6
W    2
dtype: int64

In [25]:
100*df_in_bounds.sum()/14

A    28.571429
B    42.857143
C    28.571429
D     7.142857
E    21.428571
F    35.714286
G    42.857143
H    14.285714
I    35.714286
J    21.428571
K    14.285714
L    28.571429
M    35.714286
N    35.714286
O    42.857143
P    28.571429
Q    21.428571
R    42.857143
S    50.000000
T    21.428571
U    42.857143
V    42.857143
W    14.285714
dtype: float64

In [26]:
df_in_bounds.sum().sum()

98

In [27]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [28]:
df_prediction.columns

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X'],
      dtype='object')

In [29]:
mean_squared_error(df_prediction, df_test)

1717365.3962309968

In [30]:
mean_absolute_error(df_prediction_avg, df_test)

234.94541964285713

In [31]:
mean_absolute_percentage_error(df_prediction_avg, df_test)

0.3423358065971375

In [32]:
df_prediction_avg.to_excel("Test_forecast_avg.xlsx")
lower.to_excel("Test_forecast_lower.xlsx")
upper.to_excel("Test_forecast_upper.xlsx")
df_true.to_excel("True_values.xlsx")

In [34]:
df_somme = pd.DataFrame(index = df_prediction_avg.columns)

In [35]:
df_somme["Tot. forecast"] = df_prediction_avg.sum()
df_somme["Tot. true"] = df_test.sum()
df_somme["Diff. (absolute value)"] = abs(df_somme["Tot. forecast"] - df_somme["Tot. true"])
df_somme["Diff. %"] = (100 * df_somme["Diff. (absolute value)"]) / df_somme["Tot. true"]
df_somme

,Tot. forecast,Tot. true,Diff. (absolute value),Diff. %
A,2657.9,2955.000,297.100,10.054146
B,228.7,243.000,14.300,5.884774
C,1027.5,1113.000,85.500,7.681941
D,14690.0,15829.780,1139.780,7.200226
E,91420.8,101125.339,9704.539,9.596545
F,237388.6,253818.000,16429.400,6.472906
G,256.2,252.000,4.200,1.666667
H,2408.6,2546.000,137.400,5.396701
I,999.5,1144.000,144.500,12.631119
J,1575.6,1591.000,15.400,0.967945


In [36]:
df_somme["Diff. %"].mode()

0      0.967945
1      1.666667
2      1.909548
3      2.347826
4      2.586595
5      2.698413
6      4.428571
7      5.396701
8      5.764706
9      5.884774
10     6.472906
11     7.200226
12     7.681941
13     9.596545
14    10.054146
15    10.408719
16    10.612245
17    11.331269
18    12.332215
19    12.631119
20    16.967742
21    23.360656
22    28.768116
23    55.238095
dtype: float64

In [37]:
df_somme.to_excel("residuals.xlsx")